# Gene matrix recipe

### 0. Preparation

In [2]:
# Import statements
import os
import yaml
import numpy as np
import pandas as pd

In [5]:
# Path settings
project_dir = os.path.dirname(os.path.abspath('.'))
conf_dir = os.path.join(project_dir, 'conf')
path_conf_path = os.path.join(conf_dir, 'filepaths.yaml')
gene_list_conf_path = os.path.join(conf_dir, 'gene_list_names.yaml')
gene_type_conf_path = os.path.join(conf_dir, 'gene_biotype.yaml')

with open(path_conf_path) as path_conf_file:
    path_dict = yaml.safe_load(path_conf_file)
    
with open(gene_list_conf_path) as gene_list_conf_file:
    gene_list_name_dict = yaml.safe_load(gene_list_conf_file)

gencode_path = os.path.join(project_dir, path_dict['GENCODE'])
prev_gene_mat_path = os.path.join(project_dir, path_dict['An2018'])
hgnc_path = os.path.join(project_dir, path_dict['HGNC'])
alt_gene_list_path = os.path.join(project_dir, path_dict['An2018_ALT_GENE'])

In [6]:
# Load the data
gencode_df = pd.read_table(gencode_path, compression='gzip', comment='#', names=['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attribute'])  
gencode_gene_df = gencode_df[gencode_df['feature'] == 'gene']  # List up only genes
gencode_tx_df = gencode_df[gencode_df['feature'] == 'transcript']  # List up only transcripts
hgnc_df = pd.read_table(hgnc_path, usecols=['hgnc_id', 'symbol', 'alias_symbol', 'prev_symbol', 'ensembl_gene_id'])
prev_gene_mat_df = pd.read_excel(prev_gene_mat_path, sheet_name='8-1 Genesets')

In [7]:
# Function to parse strings of the 'attribute' field in the GENCODE.
def parse_attr_str(attr_str):
    attrs = attr_str.split(';')
    attr_dict = {}
    
    for attr in attrs:
        key, value = attr.split('=')
        attr_dict[key] = value
        
    return attr_dict

In [8]:
# Parse values of the 'attribute' field in the GENCODE
gene_to_attr_dict = {}  # Key: GeneID, Value: A dictionary for the information in the 'attribute' columns

for attr_str in gencode_gene_df['attribute'].values:
    attr_dict = parse_attr_str(attr_str)
    
    if not attr_dict['ID'].endswith('Y'):  # Ignore pseudoautosome region (PAR_Y)
        gene_id = attr_dict['ID'].split('.')[0]
        gene_to_attr_dict[gene_id] = attr_dict

tx_to_attr_dict = {}  # Key: GeneID, Value: A dictionary for the information in the 'attribute' columns

for attr_str in gencode_tx_df['attribute'].values:
    attr_dict = parse_attr_str(attr_str)
    
    if not attr_dict['ID'].endswith('Y'):  # Ignore pseudoautosome region (PAR_Y)
        tx_id = attr_dict['ID'].split('.')[0]
        tx_to_attr_dict[tx_id] = attr_dict

### 1. Make a list of alternative gene names and IDs 
The purpose of this step is to find deprecated genes from the *8-1 Genesets* sheet of the **Supplementary Table S8** in An *et al.*, *Science*, 2018 (This is the **previous gene matrix**).
The reference of the gene list is GENCODE v33.

*Note: If you alreday have the file of {alt_gene_list_path}, skip this step.*

In [6]:
# Calculate No. of not available genes in the previous gene matrix
gene_id_set = set(gene_to_attr_dict.keys())
prev_gene_ids = np.vectorize(lambda x: x.split('.')[0])(prev_gene_mat_df['EnsemblGeneId'].values)
is_depr_gene = np.vectorize(lambda x: x not in gene_id_set)(prev_gene_ids)
print(f'No. deprecated genes: {sum(is_depr_gene)}')
depr_gene_mat_df = prev_gene_mat_df[is_depr_gene]
depr_gene_mat_df.sum(0)

No. deprecated genes: 482


Genes                             AC000041.2AC002310.4AC002384.2AC002465.1AC0024...
EnsemblGeneId                     ENSG00000274457.1ENSG00000260869.1ENSG00000283...
ASD associated genes (FDR≤0.1)                                                    0
ASD associated genes (FDR≤0.3)                                                    0
Midfetal co-expression genes                                                      0
Brain expressed genes                                                            11
Constrained genes (pLI ≥ 0.9)                                                     0
Postsynaptic density genes                                                        1
Developmental delay genes                                                         2
CHD8 targets                                                                      0
FMRP targets                                                                      0
Protein Coding                                                              

In [8]:
# The list of not available genes with their alternative names and IDs
# The genes in the gene lists except GENCODE biotypes and results of An et al., 2018 were considered.
# Columns: gene, gene_id, alt_gene, alt_gene_id
# Ref: https://www.genecards.org, http://asia.ensembl.org
na_gene_entries = [
    ['AKAP2', 'ENSG00000241978.9', 'PALM2AKAP2', 'ENSG00000157654.19'],
    ['BTBD8', 'ENSG00000284413.2', 'BTBD8', 'ENSG00000189195.13'],
    ['C2orf48', 'ENSG00000163009.8', 'RRM2', 'ENSG00000171848.15'],
    ['C3orf36', 'ENSG00000221972.3', 'SLCO2A1', 'ENSG00000174640.15'],
    ['C8orf44', 'ENSG00000213865.7', 'SGK3', 'ENSG00000104205.15'],
    ['C9orf47', 'ENSG00000186354.10', 'S1PR3', 'ENSG00000213694.5'],
    ['HIST1H3B', 'ENSG00000274267.1', 'H3C2', 'ENSG00000286522.1'],
    ['HIST1H3C', 'ENSG00000278272.1', 'H3C3', 'ENSG00000287080.1'],
    ['SCO2', 'ENSG00000130489.14', 'SCO2', 'ENSG00000284194.2'],
    ['TBCE', 'ENSG00000116957.12', 'TBCE', 'ENSG00000284770.2'],
    ['TMEM133', 'ENSG00000170647.3', 'ARHGAP42', 'ENSG00000165895.19'],
    ['PAML2', 'ENSG00000243444.7', 'PALM2AKAP2', 'ENSG00000157654.19'], 
]

# Write the entries
with open(alt_gene_list_path, 'w') as alt_gene_list_file:
    print('gene', 'gene_id', 'alt_gene', 'alt_gene_id', sep='\t', file=alt_gene_list_file)
    
    for gene_entry in na_gene_entries:
        print(*gene_entry, sep='\t', file=alt_gene_list_file)

### 2. Make the previous gene matrix compatible with GENCODE v33

In [9]:
# Load the list of n/a genes in the previous gene matrix 
alt_gene_list_df = pd.read_table(alt_gene_list_path)

# Change the previous gene names and IDs into their alternatives
alt_gene_dict = {}  # Key: prev_gene_id, Value: (alt_gene_name, alt_gene_id)

for _, gene_entry in alt_gene_list_df.iterrows():
    alt_gene_dict[gene_entry['gene_id']] = (gene_entry['alt_gene'], gene_entry['alt_gene_id'])
    
gene_names = prev_gene_mat_df['Genes'].values
gene_ids = prev_gene_mat_df['EnsemblGeneId'].values

for i in range(len(gene_names)):
    prev_gene_name = gene_names[i]
    prev_gene_id = gene_ids[i]
    alt_gene = alt_gene_dict.get(prev_gene_id)
    
    if alt_gene is not None:
        alt_gene_name, alt_gene_id = alt_gene
        gene_names[i] = alt_gene_name
        gene_ids[i] = alt_gene_id

# Remove genes in pseudoautosome regions (PAR_Y)
prev_gene_mat_df = prev_gene_mat_df[np.vectorize(lambda x: not x.endswith('_PAR_Y'))(prev_gene_mat_df['EnsemblGeneId'].values)]

# Trim the redundant part of each ensemble gene ID and add them into the previous gene matrix
gene_ids = np.vectorize(lambda x: x.split('.')[0])(prev_gene_mat_df['EnsemblGeneId'].values)
prev_gene_mat_df['GeneID'] = gene_ids
prev_gene_mat_df.head()

,Genes,EnsemblGeneId,ASD associated genes (FDR≤0.1),ASD associated genes (FDR≤0.3),Midfetal co-expression genes,Brain expressed genes,Constrained genes (pLI ≥ 0.9),Postsynaptic density genes,Developmental delay genes,CHD8 targets,...,Protein Coding,mis3_pro,mis3_sib,Prom_pro,Prom_sib,Prom_ActiveTSS_pro,Prom_ActiveTSS_sib,Prom_ConservedLoci_pro,Prom_ConservedLoci_sib,GeneID
0,5_8S_rRNA,ENSG00000275877.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ENSG00000275877
1,5S_rRNA,ENSG00000201285.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ENSG00000201285
2,7SK,ENSG00000202198.1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ENSG00000202198
3,A1BG,ENSG00000121410.11,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,1,0,0,ENSG00000121410
4,A1BG-AS1,ENSG00000268895.5,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,ENSG00000268895


In [30]:
# Values from the previous gene matrix
prev_gene_mat_values = prev_gene_mat_df.values
prev_gene_mat_cols = prev_gene_mat_df.columns.values
col_idx_dict = {colname: i for i, colname in enumerate(prev_gene_mat_cols)}

# Get values from the previous gene matrix and store as dictionary
prev_gene_mat_dict = {}  # Key: gene ID, Value: a dictionary for values of one row in the previous gene matrix

for prev_gene_vals in prev_gene_mat_values:
    gene_val_dict = {}
    
    for colname in prev_gene_mat_cols:
        gene_list_name = gene_list_name_dict.get(colname)
        
        if gene_list_name is not None:  # If None, this column is not for gene lists.
            prev_gene_list_name = colname
            gene_val_dict[gene_list_name] = prev_gene_vals[col_idx_dict[prev_gene_list_name]]
    
    gene_id = prev_gene_vals[col_idx_dict['GeneID']]
    same_gene_dict = prev_gene_mat_dict.get(gene_id)
    
    if same_gene_dict is not None:
        for gene_list_name in gene_val_dict:
            gene_val_dict[gene_list_name] |= same_gene_dict[gene_list_name]
    
    prev_gene_mat_dict[gene_id] = gene_val_dict

# Make a dictionary for the new gene matrix
new_gene_mat_dict = {}

for gene_id in gene_to_attr_dict:
    gene_val_dict = prev_gene_mat_dict.get(gene_id, dict())
    gene_val_dict['gene_id'] = gene_to_attr_dict[gene_id]['ID']
    gene_val_dict['gene_name'] = gene_to_attr_dict[gene_id]['gene_name']
    new_gene_mat_dict[gene_id] = gene_val_dict

# Make a DataFrame for the new gene matrix
gene_mat_df = pd.DataFrame.from_dict(new_gene_mat_dict, orient='index')
gene_mat_cols = list(gene_mat_df.columns.values)
gene_mat_cols = gene_mat_cols[-2:] + gene_mat_cols[:-2]
gene_mat_df = gene_mat_df[gene_mat_cols]
gene_mat_df.fillna(0, inplace=True)
gene_mat_df = gene_mat_df.astype({gene_list_col: 'int64' for gene_list_col in gene_mat_cols[2:]})
gene_mat_df.head()

,gene_id,gene_name,ASD_TADA_FDR01,ASD_TADA_FDR03,ASD_midfetal_coexpression,BE,PLI90Score,PSD,DDD,CHD8_targets,FMRP_targets
ENSG00000223972,ENSG00000223972.5,DDX11L1,0,0,0,0,0,0,0,0,0
ENSG00000227232,ENSG00000227232.5,WASH7P,0,0,0,0,0,0,0,0,0
ENSG00000278267,ENSG00000278267.1,MIR6859-1,0,0,0,0,0,0,0,0,0
ENSG00000243485,ENSG00000243485.5,MIR1302-2HG,0,0,0,0,0,0,0,0,0
ENSG00000284332,ENSG00000284332.1,MIR1302-2,0,0,0,0,0,0,0,0,0


### 3. Add GENCODE biotypes as gene list columns into the gene matrix

In [33]:
# Add GENCODE biotypes
with open(gene_type_conf_path) as gene_type_conf_file:
    biotype_dict = yaml.safe_load(gene_type_conf_file)

for biotype_category in biotype_dict:
    biotype_set = set(biotype_dict[biotype_category])
    gene_mat_val_dict = {gene_id: 1 if gene_to_attr_dict[gene_id]['gene_type'] in biotype_set else 0 for gene_id in gene_to_attr_dict}
    gene_mat_df[biotype_category] = pd.Series(gene_mat_val_dict)
    
gene_mat_df.head()

,gene_id,gene_name,ASD_TADA_FDR01,ASD_TADA_FDR03,ASD_midfetal_coexpression,BE,PLI90Score,PSD,DDD,CHD8_targets,FMRP_targets,Protein_Coding,Long_ncRNA,Small_ncRNA,Pseudogene,IG_TR_Gene
ENSG00000223972,ENSG00000223972.5,DDX11L1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
ENSG00000227232,ENSG00000227232.5,WASH7P,0,0,0,0,0,0,0,0,0,0,0,0,1,0
ENSG00000278267,ENSG00000278267.1,MIR6859-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0
ENSG00000243485,ENSG00000243485.5,MIR1302-2HG,0,0,0,0,0,0,0,0,0,0,1,0,0,0
ENSG00000284332,ENSG00000284332.1,MIR1302-2,0,0,0,0,0,0,0,0,0,0,0,1,0,0


### 4. Update new gene lists
The purpose of the following steps is to update new datasets to our gene matrix.

##### 01. 102 ASD genes (Satterstrom et al., Cell, 2020)

In [37]:
# Load the dataset
asd_gene_list_path = os.path.join(project_dir, path_dict['ASD'])
asd_df = pd.read_excel(asd_gene_list_path, sheet_name='102_ASD')
asd_df.dropna(inplace=True)

# Find the 102 ASD genes from the genes of the gene matrix
asd_gene_set = set(asd_df['ensembl_gene_id'].values)
is_asd_gene = np.vectorize(lambda gene_id: 1 if gene_id in asd_gene_set else 0)(gene_mat_df.index.values)

# Update the gene list to the gene matrix
asd_colname = gene_list_name_dict['ASD']
gene_mat_df[asd_colname] = is_asd_gene
gene_mat_df.head()

##### 02. 299 DDD genes

In [44]:
# Load the dataset
ddd_gene_list_path = os.path.join(project_dir, path_dict['DDD'])
ddd_df = pd.read_excel(ddd_gene_list_path, sheet_name='kaplanis_samocha_denovoWEST_res')
ddd_df = ddd_df[ddd_df['significant'] == True]  # Leave only significant genes
ddd_df = ddd_df.astype({'hgnc_id': 'int64'})

# FInd the 299 DDD genes from the genes of the gene matrix
ddd_hgnc_ids = np.vectorize(lambda x: f'HGNC:{x}')(ddd_df['hgnc_id'].values)
ddd_hgnc_id_set = set(ddd_hgnc_ids)
gene_id_to_hgnc_id = {gene_id: gene_to_attr_dict[gene_id].get('hgnc_id') for gene_id in gene_to_attr_dict}  
is_ddd_gene = np.vectorize(lambda gene_id: 1 if gene_id_to_hgnc_id[gene_id] in ddd_hgnc_id_set else 0)(gene_mat_df.index.values)

# Update the gene list to the gene matrix
ddd_colname = gene_list_name_dict['DDD']
gene_mat_df[ddd_colname] = is_ddd_gene
gene_mat_df.head()

299


,gene_id,gene_name,ASD_TADA_FDR01,ASD_TADA_FDR03,ASD_midfetal_coexpression,BE,PLI90Score,PSD,DDD,CHD8_targets,FMRP_targets,Protein_Coding,Long_ncRNA,Small_ncRNA,Pseudogene,IG_TR_Gene,ASD102_Satterstrom2020,DDD299
ENSG00000223972,ENSG00000223972.5,DDX11L1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
ENSG00000227232,ENSG00000227232.5,WASH7P,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
ENSG00000278267,ENSG00000278267.1,MIR6859-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
ENSG00000243485,ENSG00000243485.5,MIR1302-2HG,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
ENSG00000284332,ENSG00000284332.1,MIR1302-2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


##### 03. Constrained genes

In [48]:
# load the dataset
gnomad_gene_list_path = os.path.join(project_dir, path_dict['GNOMAD_GENE'])
gnomad_gene_df = pd.read_table(gnomad_gene_list_path)

# Extract HI genes (HI: Haploinsufficient)
is_hi_gene_func = lambda pli: pli is not np.nan and pli >= 0.9
is_hi_gene = np.vectorize(is_hi_gene_func)(gnomad_gene_df['pLI'].values)
hi_gene_df = gnomad_gene_df[is_hi_gene]  

# Dictionaries for pLI scores
tx_to_pli = {}
symbol_to_pli = {}
gene_to_pli = {}

# Update the 'tx_to_pli' and the 'symbol_to_pli'
hi_txs = hi_gene_df['transcript'].values
hi_symbols = hi_gene_df['gene'].values
hi_plis = hi_gene_df['pLI'].values

for tx_id, symbol, pli_score in zip(hi_txs, hi_symbols, hi_plis):
    tx_to_pli[tx_id] = pli_score
    prev_symbol_pli = symbol_to_pli.get(symbol)
    
    # Choose the maximum pLI score for duplicated symbols.
    if prev_symbol_pli is None or prev_symbol_pli < pli_score:  
        symbol_to_pli[symbol] = pli_score

# There are 3 steps to update the 'gene_to_pli'
# Step 1: Update the 'gene_to_pli' dictionary by getting gene IDs of the transcript IDs via the GENCODE
depr_tx_set = set()
hi_tx_list = hi_gene_df['transcript'].values

for tx_id in hi_tx_list:
    attr_dict = tx_to_attr_dict.get(tx_id)
    
    if attr_dict is None:
        depr_tx_set.add(tx_id)
    else:
        gene_id = attr_dict['Parent'].split('.')[0]
        prev_gene_pli = gene_to_pli.get(gene_id)
        pli_score = tx_to_pli[tx_id]
        
        # Choose the maximum pLI score for duplicated genes.
        if prev_gene_pli is None or prev_gene_pli < pli_score:    
            gene_to_pli[gene_id] = pli_score

# Get the ensembl gene IDs of the gene symbols via the HGNC 
hgnc_df_col_idx = {column: i for i, column in enumerate(hgnc_df.columns.values)}
hgnc_df_val = hgnc_df.values

symbol_to_gene_id = {}
alias_to_gene_id = {}
prev_symbol_to_gene_id = {}

for hgnc_entry in hgnc_df_val:
    gene_symbol = hgnc_entry[hgnc_df_col_idx['symbol']]
    alias_symbol_str = hgnc_entry[hgnc_df_col_idx['alias_symbol']]
    prev_symbol_str = hgnc_entry[hgnc_df_col_idx['prev_symbol']]
    gene_id = hgnc_entry[hgnc_df_col_idx['ensembl_gene_id']]
    
    symbol_to_gene_id[gene_symbol] = gene_id
    
    if alias_symbol_str is not np.nan:
        alias_symbols = alias_symbol_str.split('|')
        
        for alias_symbol in alias_symbols:
            alias_to_gene_id[alias_symbol] = gene_id
    
    if prev_symbol_str is not np.nan:
        prev_symbols = prev_symbol_str.split('|')
        
        for prev_symbol in prev_symbols:
            prev_symbol_to_gene_id[prev_symbol] = gene_id

# Step 2: Update the 'gene_to_pli' dictionary by getting gene IDs of the deprecated transcript IDs using the gene symbols in the HGNC
depr_hi_gene_df = hi_gene_df[np.vectorize(lambda tx_id: tx_id in depr_tx_set)(hi_txs)]
depr_gene_symbols = depr_hi_gene_df['gene'].values

for depr_gene_symbol in depr_gene_symbols:
    # Priority: symbol -> alias -> previous symbol (in HGNC)
    gene_id = symbol_to_gene_id.get(depr_gene_symbol)
    
    if gene_id is None:
        gene_id = alias_to_gene_id.get(depr_gene_symbol)
        
        if gene_id is None:
            gene_id = prev_symbol_to_gene_id.get(depr_gene_symbol)
            
            if gene_id is None:
                print(f'{depr_gene_symbol} cannot be found in both the GENCODE and the HGNC.')
                continue
    
    # Update only if the same gene ID does not exist
    if gene_to_pli.get(gene_id) is None:  
        gene_to_pli[gene_id] = symbol_to_pli[depr_gene_symbol]

# Step 3: Add the alternative gene ID of 'AC005358.1' to the dictionary (This gene ID cannot be found in the above 2 steps).
# Ref: https://www.genecards.org/cgi-bin/carddisp.pl?gene=MYOCD-AS1&keywords=AC005358,1
gene_to_pli['ENSG00000227274'] = symbol_to_pli['AC005358.1']

# Update the gene matrix
# HC: High-confident
is_hi_gene = np.vectorize(lambda gene_id: 0 if gene_to_pli.get(gene_id) is None else 1)(gene_mat_df.index.values)  # pLI score >= 0.9
is_hc_hi_gene = np.vectorize(lambda gene_id: 1 if gene_to_pli.get(gene_id) is not None and gene_to_pli.get(gene_id) >= 0.995 else 0)(gene_mat_df.index.values)  # pLI score >= 0.995
gene_mat_df[gene_list_name_dict['GNOMAD_PLI90']] = is_hi_gene
gene_mat_df[gene_list_name_dict['GNOMAD_PLI995']] = is_hc_hi_gene
gene_mat_df.head()

AC005358.1 cannot be found in both the GENCODE and the HGNC.


,gene_id,gene_name,ASD_TADA_FDR01,ASD_TADA_FDR03,ASD_midfetal_coexpression,BE,PLI90Score,PSD,DDD,CHD8_targets,FMRP_targets,Protein_Coding,Long_ncRNA,Small_ncRNA,Pseudogene,IG_TR_Gene,ASD102_Satterstrom2020,DDD299,HI_GENE_pLI90_gnomAD,HI_GENE_pLI995_gnomAD
ENSG00000223972,ENSG00000223972.5,DDX11L1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
ENSG00000227232,ENSG00000227232.5,WASH7P,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
ENSG00000278267,ENSG00000278267.1,MIR6859-1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
ENSG00000243485,ENSG00000243485.5,MIR1302-2HG,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
ENSG00000284332,ENSG00000284332.1,MIR1302-2,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
